# Projeto Integrador

O primeiro passo da etapa de pré-processamento dos dados é importar as bibliotecas necessárias para essa tarefa.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from ydata_profiling import ProfileReport
import hvplot.pandas
import missingno as msno
import altair as alt

Importando o dataset do projeto:

In [4]:
df = pd.read_csv('Bases/cs_bisnode_panel.csv')

Retirando as colunas com alto indice de missing data, conforme orientado nas instruções.

In [5]:
df = df.drop(['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D'], axis=1)

Removendo dos dados os registros do ano de 2016

In [6]:
df = df[df['year']!=2016]

Verificando o tipo de dado de cada variável do dataset:

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278086 entries, 0 to 287828
Data columns (total 42 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   comp_id               278086 non-null  float64
 1   begin                 278086 non-null  object 
 2   end                   278086 non-null  object 
 3   amort                 270070 non-null  float64
 4   curr_assets           277955 non-null  float64
 5   curr_liab             277955 non-null  float64
 6   extra_exp             269300 non-null  float64
 7   extra_inc             269300 non-null  float64
 8   extra_profit_loss     270626 non-null  float64
 9   fixed_assets          277955 non-null  float64
 10  inc_bef_tax           270649 non-null  float64
 11  intang_assets         277946 non-null  float64
 12  inventories           277955 non-null  float64
 13  liq_assets            277955 non-null  float64
 14  material_exp          270070 non-null  float64
 15  

In [ ]:
df.T

Vamos converter os tipos de dados algumas colunas para que estas esteja coerentes com as descrição das variáveis, a saber:

- comp_id: como é a identificação da empresa, deve ser considerada uma string (object)
- begin: converter para data
- end: converter para data
- founded_date: converter para data
- exit_date: converter para data
- nace_main: converter para string (object)
- ind2: converter para string (object)
- ind: converter para string (object)

In [ ]:
df['comp_id'] = df['comp_id'].astype('object')
df['begin'] = pd.to_datetime(df['begin'],format="%Y-%m-%d", errors='coerce')
df['end'] = pd.to_datetime(df['end'],format="%Y-%m-%d", errors='coerce')
df['founded_date'] = pd.to_datetime(df['founded_date'],format="%Y-%m-%d", errors='coerce')
df['exit_date'] = pd.to_datetime(df['exit_date'],format="%Y-%m-%d", errors='coerce')
df['nace_main'] = df['nace_main'].astype('object')
df['ind2'] = df['ind2'].astype('object')
df['ind'] = df['ind'].astype('object')

In [ ]:
#Verificando se a conversão funcionou:
df.info()

O próxima passo é verificar a quantidade de missing values em cada coluna:

In [ ]:
msno.bar(df)

In [ ]:
missing_values = df.isna().sum()/len(df)
missing_values = pd.DataFrame({ 'Percentual_Valores_Faltantes': missing_values}).reset_index()
missing_values[missing_values['Percentual_Valores_Faltantes'] > 0]

# Tratar os missing values de Sales

Ver a distribuição por ano dos missing values

------------------------------------------

O próximo passo é ver como esses missing values serão tratados.

A variável "labor_avg" (número médio de empregados no ano) possui mais da metade dos dados faltantes. Como há um variável chamada "personnel_exp", que representa os gastos com pessoal, vamos verificar se há algum correlação entre essas duas variáveis por meio de um gráfico de dispersão:

In [ ]:
sns.lmplot(x="personnel_exp", y="labor_avg", data=df)
plt.xlabel('Custo com Pessoal')
plt.ylabel('Média de Empregados por Ano')
plt.title('Análise de Relação entre Custo com Pessoal e Média de Empregados')
plt.show()

Como há uma correlação não muito forte e com pontos fora do intervalo de confiança da reta do modelo de regressão linear para essa duas variáveis, "labor_avg" será mantida no dataset. Assim, serão preenchidos os dados faltantes com a média e avaliado durante a fase de modelagem se essa variável contribui ou prejudica o desempenho dos modelos de predição.

Para as variáveis que são categóricas e que tem missing values e podem a priori agregar pouco ao modelo ou até mesmo criar viés, uma vez que não há evidência científica que gênero ou origem influenciam na gestão de uma empresa, vamos excluí-las, a saber:

- gender
- origin
- nace_main
- ind2
- ind
- foreign
- female

Para as colunas com a data e ano de saída (exit_date e exit_year), há muitos valores faltantes e a coluna 'in_operation', que indicará se a empresa está em operação e será calcula posteriormente, já contém essa informação. Assim, vamos excluí-la também.

In [ ]:
df = df.drop(['gender', 'origin', 'nace_main', 'ind2', 'ind', 'exit_date', 'exit_year', 'foreign', 'female'], axis=1)

Para as colunas numéricas, vamos substituir os valores faltantes pela média do valor da variável calculado por empresa, e não a média global da variável, visando manter a distribuição dos dados no nível da empresa. Caso a empresa não possua valores da variável em questão, vamos substituir por ZERO, uma vez que não há informações suficiente para definir esses valores sem causar um grande correlação com outras variáveis.

O primeiro passo é criar um dataframe com os valores das médias por empresa.

In [ ]:
#Criando dataframe com a média das variáveis numéricas por empresa.
media_variavel = df.groupby('comp_id').agg({'material_exp': 'mean', 'personnel_exp': 'mean',
                                             'profit_loss_year': 'mean', 'sales': 'mean', 'share_eq': 'mean' , 'labor_avg': 'mean'})

#Caso haja algum empresa em valor em quaisquer anos, consideraremos ZERO o valor da variável:
media_variavel = media_variavel.fillna(0)

#Ajustando o nome das colunas para facilitar o merge com o dataframe principal (incluindo 'Mean_' no início de todas, exceto 'comp_id'
nome_coluna = []
for coluna in list(media_variavel.columns):
    nome_coluna.append('Mean_'+coluna)
media_variavel.columns = nome_coluna
media_variavel = media_variavel.reset_index()
media_variavel.head()

A partir do dataframe com as médias das variáveis por empresa, vamos preencher os valores faltantes das colunas numéricas do dataframe principal com essas médias.

In [ ]:
#Vamos usar uma lista com os nomes das colunas para iterar e facilitar o preenchimento
for coluna in list(media_variavel.columns)[1:]: 
    # mescla o df principal com o df com médias (o nome da coluna desse df começa com Mean)
    df = pd.merge(df, media_variavel[['comp_id',coluna]], how = 'left', on = ['comp_id'])
    #preenche os dados faltantes com a média por empresa da variável
    df[coluna[5:]] = df[[coluna[5:],coluna]].apply(lambda x : x[coluna] if pd.isna(x[coluna[5:]]) else x[coluna[5:]], axis = 1)
    #Remove a coluna com o nome Mean
    df = df.drop([coluna], axis=1)

# Verifica se o código acima foi efetivo
msno.bar(df)

Com os variáveis numéricas sem dados faltantes, restam as das variáveis categóricas, datas e binárias. Para as variáveis "founded_date", vamos usar a moda (valor mais frequente), pois os valores faltantes nessa coluna representam menos de 0,1% e se trata de uma variável categórica.

In [ ]:
df["founded_date"].fillna(df["founded_date"].mode()[0], inplace = True)
#Verificando se todas as linhas dessa coluna foram preenchidas.
df["founded_date"].isnull().sum()

Vamos analisar a coluna 'region_m'e os seus dados faltantes:

In [ ]:
df['region_m'].value_counts()

In [ ]:
df['region_m'].describe()

In [ ]:
# Missing values
print(df['region_m'].isna().sum())
print(df['region_m'].isna().sum()/len(df))

Os dados faltantes da coluna 'region_m' representa menos de 0,3% dos valores da coluna (800 obseervações). Assim como a variável "founded_date", vamos usar a moda nessa colunba:

In [ ]:
df['region_m'].fillna(df['region_m'].mode()[0], inplace = True)
#Verificando se todas as linhas dessa coluna foram preenchidas.
df['region_m'].isnull().sum()

O passo seguinte é criar uma nova variável que verifica se a empresa estava operando ou não. O critério é verificar se a empresa teve faturamento X+2. Essa será a variável que deverá ser predita (variável resposta).

Como iremos criar uma coluna para verificar se a empresa estava operando ou não a partir dos dados da coluna 'sales', verificaremos a qualidade das informações dessa última coluna:

In [ ]:
#Vamos dividir a coluna de 'sales' por 1 milhão para facilitar a visualização
df_MM = df['sales']/1000000
df_MM.describe()

Esse coluna possui valores negativos, o que não faz sentido para faturamento em 1 ano. Nesse caso, consideraremos os valores negativos como ZERO.

In [ ]:
df['sales'] = df['sales'].apply(lambda x: 0 if x < 0 else x)

Verificando se o código acima funcionou:

In [ ]:
df_MM = df['sales']/1000000
df_MM.describe()

Como o objetivo é predizer se uma empresa deicará de operar em até dois anos, vamos criar a partir dos dados da coluna 'sales' uma nova coluna para verificar se a empresa estava operando ou não no final do período, que será a variável resposta dos modelos. O primeiro passo é criar um dataframe somente com os valores de faturamento por ano e empresa. Como o objetivo é ver se a empresa estava operando ao final do período e vamos utilizar os dados de 2012 para estimar os modelos de predição, os dois últimos anos devem ser zero. Vamos considerar para a análise somente esses dois anos.

In [ ]:
#Criando um um dataframe somente com os anos de 2014 e 2015
df_sales = df[(df['year']==2014) & (df['year']==2015)]
#criando um dataframe que soma os dados de faturamento cada empresa nesses dois anos.
df_sales = pd.DataFrame(df.groupby(["comp_id"])['sales'].sum()).reset_index()
df_sales

Assim, as empresas que tiveram faturamento zero em 2014 e 2015 deve ser consideradas como fora de operação. Vamos criar a coluna com a variável resposta com essas informações:

In [ ]:
df_sales['will_close'] = df_sales['sales'].apply(lambda x: 1 if x == 0 else 0)
df_sales

Vamos contar quantas empresas estavam operando:

In [ ]:
df_sales['will_close'].value_counts()/df_sales['will_close'].count()*100

Cerca de 8,84% das empresas não estão mais operando.

Vamos juntar os dados da coluna que diz se a empresa estava em operação ('in_operation') ao dataframe inicial:

In [ ]:
df = pd.merge(df, df_sales[['comp_id','will_close']], how = 'left', on = ['comp_id'])
df.info()

O próximo passo é criar uma coluna com a idade da empresa:

In [ ]:
# Criando a coluna com a idade da empresa
df['company_age'] = df['year'] - df['founded_year']

Agora, vamos filtrar os dados de 2012 e manter no dataframe somente as empresas com faturamento abaixo de 10 milhões e acima de 1000 euros.

In [ ]:
#Filtro para o ano de 2012
df = df[df['year']==2012]

In [ ]:
# Filtro para as empresas com faturamento abaixo de 10 milhões e acima de 1000 euros
df = df[(df['sales']<10000000) | (df['sales']>1000)]

In [ ]:
df.head()

O próximo passo é criar a coluna por porte de faturamento. A União Europeia define como micro empresas aqueles com faturamento até 2 milhões de euros por ano, enquanto as pequenas e as médias tem que ter uma receita de até 10 milhões e 50 milhões, respectivamente. Como foi retirada da base os faturamentos acima de 10 milhões de euros, esse corte cria somente dois grupos. Assim, vamos analisar a distribuição de faturamento no dataset em questão.

In [ ]:
#Vamos usar a pandas series criada com o faturamento na escala de milhão
sns.histplot(data=df_MM, bins=100)
plt.xlabel('Faturamento em Milhão de Euros')
plt.ylabel('Contagem')
plt.title('Distribuição de Faturamento - Milhão de Euros')
plt.show()

De acordo com o histograma acima, que foi dividido em grupos de 1 milhão de euros, a grande maioria das empresas têm faturamento abaixo de 1 milhão em 2012 (variável assimétrica). Assim, para que na modelagem não haja grupos com baixo número de observações, vamos dividir em 2 grupos. Para tal, será aplicado o seguinte critério:

- pequena empresa: faturamento menor do que 50 mil
- média e grande empresa: faturamento acim 50 mil

In [ ]:
# Criando a coluna com o porte da empresa por faturamento
df['company_size'] = df['sales'].apply(lambda x: 'small' if x <= 50000 else 'large_medium')
df['company_size'].value_counts()

Vamos plotar os histogramas por porte de empresa:

In [ ]:
# Criar uma figura com dois subplots (quadrados diferentes)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plotar o primeiro histograma na primeira subplot
sns.histplot(data=df[df['company_size'] == 'small'], x='sales', bins=20, kde=True, ax=axes[0])
axes[0].set_xlabel('Faturamento')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Pequenas Empresas (< 100 mil euros)')

# Plotar o segundo histograma na segunda subplot
sns.histplot(data=df[df['company_size'] == 'large_medium'], x='sales', bins=20, kde=True, ax=axes[1])
axes[1].set_xlabel('Faturamento')
axes[1].set_ylabel('Frequência')
axes[1].set_title('Médias e Grandes Empresas (> 100 mil euros)')

# Ajustar o layout para evitar sobreposição
plt.tight_layout()

# Mostrar os gráficos
plt.show()


Mesmo assim, a variável parece muito assimétrica. Para mitigar o problema, vamos aplicar calcular o logaritmo dessa variável.

In [ ]:
df['sales_log'] = df['sales'].apply(lambda x: np.log(x) if x > 0 else 0)

Plotando o gráfico dessa variável, vemos que a distribuição ficou mais parecida com a normal, com alguma concentração de valores em ZERO (empresas que supostamente fecharam).

In [ ]:
sns.histplot(data=df['sales_log'], bins=10)
plt.xlabel('Logaritmo do Faturamento')
plt.ylabel('Contagem')
plt.title('Distribuição de Logaritmo do Faturamento')
plt.show()

Esse problema ocorre com as demais variáveis numéricas? Vamos usar boxplot para verificar a distribuição desses valores.
Vamos criar um dataframe somente com as variáveis numéricas.

In [ ]:
var_num = df.select_dtypes( include=['int64', 'float64'] )
var_cat = df.select_dtypes( exclude=['int64', 'float64','datetime64[ns]'] )

var_num.columns

In [ ]:
# fazendo boxplot de todas as variáveis numéricas para identificar as distribuições:

colunas = var_num.columns

# Configurações para a disposição dos subplots
num_linhas = 5  
num_colunas = 4  

# Calcula o número total de figuras necessárias
num_total_figuras = (len(colunas) // (num_linhas * num_colunas)) + 1

# Tamanho da figura
tamanho_figura = (18, 12)

# Loop para criar cada figura com os boxplots
for figura_num in range(num_total_figuras):
    fig, axs = plt.subplots(num_linhas, num_colunas, figsize=tamanho_figura)
    
    # Colunas para esta figura
    inicio_coluna = figura_num * num_linhas * num_colunas
    fim_coluna = (figura_num + 1) * num_linhas * num_colunas
    colunas_figura = colunas[inicio_coluna:fim_coluna]

    # Iterar pelas colunas e criar os boxplots para esta figura
    for i, coluna in enumerate(colunas_figura):
        linha = i // num_colunas
        col = i % num_colunas
        sns.boxplot(x=var_num[coluna], ax=axs[linha, col], width=0.5)
        axs[linha, col].set_title(coluna)
        axs[linha, col].set_xticks([])  # Remove os rótulos do eixo x

    # Ajustar o layout
    plt.tight_layout()

    # Exibir a figura
    plt.show()

In [ ]:
df_flavio = df[['material_exp', 'personnel_exp','profit_loss_year', 'sales', 'share_eq', 'labor_avg']]

In [ ]:
df_flavio.describe()

In [ ]:
df['sales_log'] = df['sales'].apply(lambda x: np.log(x) if x > 0 else 0)

In [ ]:
df = df.drop(['begin', 'end', 'year', 'founded_year', 'balsheet_flag', 'balsheet_length', 'balsheet_notfullyear',
              "ceo_count", "birth_year", "inoffice_days", 'sales_log','founded_date'  ], axis=1)

In [ ]:
df.shape

In [ ]:
df.to_csv('dados_bisnode.csv', index=False)

In [ ]:
profile = ProfileReport(df, title='Analise Exploratoria do Projeto Integrador')
profile.to_file(output_file="analise_projeto_integrador.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

In [83]:
%%html
<iframe src="analise_projeto_integrador.html" width="100%" height="800px"></iframe>

# Proximos passos

## Qualidade
ver as inconsistencias (valores negativos etc) das variaveis  e aplicar o log para variáveis assimétricas
